# Test out the pick em game before moving to its own .exe or .py

#### 2024-12-11

In [1]:
import json
import pandas as pd
import os
import random

## Load data

In [2]:
path_moves_per_pokemon = os.path.join("..", "..", "Scraping", "data", "moves_per_pokemon_orig_names.json")
path_pokemon_main_stats = os.path.join("..", "..", "Scraping", "data", "pokemon_main_stats_data.csv")
path_pokemon_moves_master = os.path.join("..", "..", "Scraping", "data", "pokemon_moves_data.csv")
path_type_effectiveness = os.path.join("..", "data", "type_effectiveness.json")

In [3]:
# Load moves per pokemon as dict
with open(path_moves_per_pokemon, 'r') as f:
    dict_moves_per_pokemon = json.load(f)

# Load type effectiveness as a dict
with open(path_type_effectiveness, 'r') as f:
    dict_type_effectiveness = json.load(f)

In [4]:
# Load dfs
## Each pokemon and their stats
df_pokemon_main_stats = pd.read_csv(path_pokemon_main_stats)

# Pokemon moves
df_pokemon_moves_master = pd.read_csv(path_pokemon_moves_master)
df_pokemon_moves_master['Accuracy'] = df_pokemon_moves_master['Accuracy'].replace('∞', '100').astype(float) # Replace '∞' with 100.0 and convert the column to floats


## Random Selection

### Randomly select two pokemon without replacement and four of their moves without replacement

In [5]:
# Randomly select two Pokémon without replacement
selected_pokemon = random.sample(list(dict_moves_per_pokemon.keys()), 2)

In [6]:
# Select 4 random moves for each Pokémon
battle_pokemon = {
    pokemon: random.sample(dict_moves_per_pokemon[pokemon], 4)
    for pokemon in selected_pokemon
}

In [7]:
battle_pokemon

{'Aromatisse': ['Return', 'Swagger', 'Echoed Voice', 'Encore'],
 'Chi-Yu': ['Snarl', 'Lash Out', 'Sleep Talk', 'Dark Pulse']}

## Pokemon battle simulation

In [14]:
def pokemon_battle(battle_pokemon, df_pokemon_main_stats, df_pokemon_moves_master, dict_type_effectiveness):
    # Initialize Pokemon with their stats (same as before)
    pokemon = {}
    for name, moves in battle_pokemon.items():
        stats = df_pokemon_main_stats[df_pokemon_main_stats['Name'] == name].iloc[0]
        pokemon[name] = {
            'hp': stats['HP'],
            'max_hp': stats['HP'],
            'attack': stats['Attack'],
            'defense': stats['Defense'],
            'sp_atk': stats['Sp. Atk'],
            'sp_def': stats['Sp. Def'],
            'speed': stats['Speed'],
            'type': stats['Type'],
            'moves': moves
        }
    
    # Determine order based on Speed
    order = sorted(pokemon.keys(), key=lambda x: pokemon[x]['speed'], reverse=True)
    
    def calculate_damage(attacker, defender, move_data, effectiveness):
        power = move_data['Power'] if not pd.isna(move_data['Power']) else 0
        
        # Determine if it's a critical hit (1/24 chance)
        is_critical = random.random() < 1/24
        critical_multiplier = 2 if is_critical else 1
        
        if move_data['Category'] == 'Physical':
            damage = (((2 * 50 / 5 + 2) * power * attacker['attack'] / defender['defense']) / 50 + 2) * effectiveness * critical_multiplier
        elif move_data['Category'] == 'Special':
            damage = (((2 * 50 / 5 + 2) * power * attacker['sp_atk'] / defender['sp_def']) / 50 + 2) * effectiveness * critical_multiplier
        else:
            damage = 0
        
        damage = max(1, int(damage))  # Ensure at least 1 damage
        return damage, is_critical
    
    # Add a 'status' field to each Pokemon
    for name in pokemon:
        pokemon[name]['status'] = None

    # Battle loop
    while all(pokemon[p]['hp'] > 0 for p in pokemon):
        for attacker_name in order:
            if pokemon[attacker_name]['hp'] <= 0:
                continue
            
            defender_name = [p for p in pokemon if p != attacker_name][0]

            # Check if the attacker is flinching
            if pokemon[attacker_name]['status'] == 'flinching':
                print(f"{attacker_name} flinched and couldn't move!")
                pokemon[attacker_name]['status'] = None
                continue            
            
            # Select a random move
            move = random.choice(pokemon[attacker_name]['moves'])
            move_data = df_pokemon_moves_master[df_pokemon_moves_master['Move Name'] == move].iloc[0]
            
            print(f"\n{attacker_name} used {move} on {defender_name}!")
            
            # Check if the move hits based on accuracy
            accuracy = move_data['Accuracy']
            if pd.isna(accuracy) or random.random() * 100 < accuracy:
                # Move hits
                # Calculate type effectiveness
                effectiveness = 1
                for att_type in pokemon[attacker_name]['type']:
                    for def_type in pokemon[defender_name]['type']:
                        if att_type in dict_type_effectiveness and def_type in dict_type_effectiveness[att_type]:
                            effectiveness *= dict_type_effectiveness[att_type][def_type]

                
                # Calculate damage
                damage, is_critical = calculate_damage(pokemon[attacker_name], pokemon[defender_name], move_data, effectiveness)
                
                # Apply damage
                pokemon[defender_name]['hp'] = max(0, pokemon[defender_name]['hp'] - damage)
                
                # Print battle text
                if is_critical:
                    print("It's a critical hit!")
                if effectiveness > 1:
                    print("It's super effective!")
                elif effectiveness < 1 and effectiveness > 0:
                    print("It's not very effective...")
                elif effectiveness == 0:
                    print("It doesn't affect " + defender_name + "...")
                
                print(f"It dealt {damage} damage. {defender_name}'s HP: {pokemon[defender_name]['hp']}/{pokemon[defender_name]['max_hp']}")


                # Apply move effect
                battle_state = {'pokemon': pokemon}
                battle_state = apply_move_effect(move_data, pokemon[attacker_name], pokemon[defender_name], battle_state)
                pokemon = battle_state['pokemon']
                
                if pokemon[defender_name]['hp'] <= 0:
                    print(f"{defender_name} fainted! {attacker_name} wins!")
                    return attacker_name
            else:
                # Move misses
                print(f"But it missed!")
    
    return "Draw"  # This should never happen in a 1v1 battle, but just in case

In [15]:
pokemon_battle(battle_pokemon, df_pokemon_main_stats, df_pokemon_moves_master, dict_type_effectiveness)


Chi-Yu used Dark Pulse on Aromatisse!
It dealt 55 damage. Aromatisse's HP: 46/101


KeyError: 'name'

In [ ]:
## Now work on:
# PP
# Effects
# Status effects
# Simplified? damage calc - what are we missing?
"""
Note that this is a simplified version and doesn't include all Pokémon battle mechanics 
 (such as status effects, critical hits, or move PP). 
 You can expand on this base to add more complex mechanics as needed.
"""

# 12/12/2024:
# Added in crit chance 
# Added in accuracy
# Added in text for crit, super/not very effective when applicable

In [25]:
df_pokemon_moves_master[df_pokemon_moves_master['Move Name'] == 'Self-Destruct']

,Move Name,Type,Category,Power,Accuracy,PP,Effect,Probability (%)
703,Self-Destruct,Normal,Physical,200.0,100.0,5.0,User faints.,NaN


In [12]:
df_pokemon_moves_master[df_pokemon_moves_master['Move Name'] == 'Guillotine']

,Move Name,Type,Category,Power,Accuracy,PP,Effect,Probability (%)
357,Guillotine,Normal,Physical,NaN,30.0,5.0,"One-Hit-KO, if it hits.",NaN


In [18]:
df_pokemon_moves_master[df_pokemon_moves_master['Move Name'] == 'Thief']

,Move Name,Type,Category,Power,Accuracy,PP,Effect,Probability (%)
849,Thief,Dark,Physical,60.0,100.0,25.0,Also steals opponent's held item.,NaN


In [5]:
df_pokemon_moves_master

,Move Name,Type,Category,Power,Accuracy,PP,Effect,Probability (%)
0,"10,000,000 Volt Thunderbolt",Electric,Special,195.0,NaN,1.0,Pikachu-exclusive Z-Move. High critical hit ra...,NaN
1,Absorb,Grass,Special,20.0,100.0,25.0,User recovers half the HP inflicted on opponent.,NaN
2,Accelerock,Rock,Physical,40.0,100.0,20.0,User attacks first.,NaN
3,Acid,Poison,Special,40.0,100.0,30.0,May lower opponent's Special Defense.,10.0
4,Acid Armor,Poison,Status,NaN,NaN,20.0,Sharply raises user's Defense.,NaN
...,...,...,...,...,...,...,...,...
929,Yawn,Normal,Status,NaN,NaN,10.0,Puts opponent to sleep in the next turn.,NaN
930,Zap Cannon,Electric,Special,120.0,50.0,5.0,Paralyzes opponent.,100.0
931,Zen Headbutt,Psychic,Physical,80.0,90.0,15.0,May cause flinching.,20.0
932,Zing Zap,Electric,Physical,80.0,100.0,10.0,May cause flinching.,30.0


In [6]:
df_pokemon_moves_master.Effect.unique()

array(['Pikachu-exclusive Z-Move. High critical hit ratio.',
       'User recovers half the HP inflicted on opponent.',
       'User attacks first.', "May lower opponent's Special Defense.",
       "Sharply raises user's Defense.", 'Poison type Z-Move.',
       "Sharply lowers opponent's Special Defense.",
       'Stronger when the user does not have a held item.',
       'Sharply raises a random stat.',
       'Ignores Accuracy and Evasiveness.', 'High critical hit ratio.',
       'Gives target priority in the next turn.',
       "Sharply raises user's Speed.", 'May cause flinching.',
       'Fighting type Z-Move.', nan,
       'User switches with opposite teammate.',
       "Sharply raises user's Special Defense.",
       'The user entangles the target with its anchor chain while attacking. The target becomes unable to flee.',
       "May raise all user's stats at once.",
       "Lowers target's Special Defense.",
       'Restores a little HP each turn.', "Raises user's Speed.",
    

In [10]:
df_pokemon_moves_master[df_pokemon_moves_master['Effect'] == 'May cause flinching.']

,Move Name,Type,Category,Power,Accuracy,PP,Effect,Probability (%)
14,Air Slash,Flying,Special,75.0,95.0,15.0,May cause flinching.,30.0
33,Astonish,Ghost,Physical,30.0,100.0,15.0,May cause flinching.,30.0
60,Bite,Dark,Physical,60.0,100.0,25.0,May cause flinching.,30.0
77,Bone Club,Ground,Physical,65.0,85.0,20.0,May cause flinching.,10.0
155,Dark Pulse,Dark,Special,80.0,100.0,15.0,May cause flinching.,20.0
194,Dragon Rush,Dragon,Physical,100.0,75.0,10.0,May cause flinching.,20.0
230,Extrasensory,Psychic,Special,80.0,100.0,20.0,May cause flinching.,10.0
246,Fiery Wrath,Dark,Special,90.0,100.0,10.0,May cause flinching.,20.0
270,Floaty Fall,Flying,Physical,90.0,95.0,15.0,May cause flinching.,30.0
369,Headbutt,Normal,Physical,70.0,100.0,15.0,May cause flinching.,30.0


### Apply Move Effects

In [13]:
def apply_move_effect(move, user, target, battle_state):
    effect = move['Effect']
    probability = move['Probability (%)']
    
    if effect == 'May cause flinching.':
        if random.random() * 100 < probability:
            target['status'] = 'flinching'
            print(f"{target['name']} is flinching!")
    
    # We'll add more effects here later
    
    return battle_state

In [11]:
random.random()

0.2446692869333028

In [12]:
random.random() * 100

67.10474017558111